# Proyecto 1 : Primera Parte 

### Integrantes :
- Carlos Julio Pinto Rodriguez 
- Juan Camilo Gonzales 

#### 1. Importación de librerias a usar

In [107]:
#!pip install inflect
#!pip install num2words


In [108]:
#pip install nltk spacy 

In [109]:
#Immporta la flexiones del idioma
#import inflect
# librería Natural Language Toolkit, usada para trabajar con textos 
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import sys
# from pandas_profiling import ProfileReport
import re, string, unicodedata
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, accuracy_score
from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt
import num2words
from nltk.stem import SnowballStemmer
import spacy


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oipli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\oipli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oipli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
# Configuring pandas to show all cell content
pd.set_option("display.max_colwidth", None)

In [111]:
nltk.download("punkt")
nltk.download("stopwords")
stop_words = stopwords.words("spanish")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oipli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\oipli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [112]:
stop_words

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

### Carga de datos, Entendimiento y Perfilamiento de datos

In [113]:
 #Carga datos
df_datos= pd.read_excel("./data/cat_345.xlsx")

In [114]:
# Asignación a una nueva variable de los datos leidos
texto_df=df_datos   

In [115]:
textos = texto_df.copy()
textos['Conteo'] = [len(x) for x in textos['Textos_espanol']]
#textos['Moda'] = textos['Textos_espanol'].apply(lambda x: max([len(word) for word in x.split(' ')]))
textos['Moda'] = textos['Textos_espanol'].apply(lambda x: max(set(x.split(' ')), key=x.split(' ').count))

textos['Max'] = textos['Conteo'].max()
textos['Min'] = textos['Textos_espanol'].apply(lambda x: min([len(word) for word in x.split(' ')]))


In [116]:
#ProfileReport(textos)

### Preparación de datos  ALTERNATIVA 1

In [117]:
# Limpieza de datos 
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    #p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            #new_word = p.number_to_words(word)
            new_word = num2words.num2words(word, lang='es')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    return [word.lower() for word in words]

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    return [word for word in words if word not in stop_words]


def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

In [118]:
#TOKENIZACION

textos['words'] = textos['Textos_espanol'].apply(word_tokenize).apply(preprocessing) #Aplica la eliminación del ruido
textos.head()

,Textos_espanol,sdg,Conteo,Moda,Max,Min,words
0,"Por ejemplo, el nÃºmero de consultas externas de especialistas es de 319 por cada mil derechohabientes en el SP, en comparaciÃ³n con 338 y 620 por cada mil derechohabientes en el IMSS y el ISSSTE, respectivamente. Si bien algunas de estas diferencias pueden reflejar una necesidad desigual (como la poblaciÃ³n ligeramente mayor del ISSSTE), otras no pueden justificarse de esta manera. El nÃºmero de recetas que no pudieron ser surtidas en su totalidad por un farmacÃ©utico debido a la falta de existencias es de 33% dentro del SP en comparaciÃ³n con 14% dentro del IMSS segÃºn los datos de la encuesta (aunque las propias cifras de los institutos de la SS sugieren tasas mÃ¡s altas de recetas surtidas). Ambas cifras se encuentran entre las mÃ¡s altas de la OCDE. El gasto de bolsillo no se ha reducido significativamente en la última década, a pesar de los esfuerzos para lograr la cobertura sanitaria universal a través de la reforma del SP.",3,944,de,1513,1,"[ejemplo, naomero, consultas, externas, especialistas, trescientos diecinueve, cada, mil, derechohabientes, sp, comparacia3n, trescientos treinta y ocho, seiscientos veinte, cada, mil, derechohabientes, imss, issste, respectivamente, si, bien, diferencias, pueden, reflejar, necesidad, desigual, poblacia3n, ligeramente, mayor, issste, pueden, justificarse, manera, naomero, recetas, pudieron, ser, surtidas, totalidad, farmacautico, debido, falta, existencias, treinta y tres, dentro, sp, comparacia3n, catorce, dentro, imss, segaon, datos, encuesta, aunque, propias, cifras, institutos, ss, sugieren, tasas, mas, altas, recetas, surtidas, ambas, cifras, encuentran, mas, altas, ocde, gasto, bolsillo, reducido, significativamente, ultima, decada, pesar, esfuerzos, lograr, cobertura, sanitaria, universal, traves, reforma, sp]"
1,"En 2007, el gobierno central financió directamente solo el 0,3% del gasto total en salud, pero realizó transferencias específicas para el gasto en salud que ascendieron a otro 5,6% del gasto total. La mayor parte del gasto presupuestario en salud lo realizan los gobiernos de los condados. Por lo tanto, los condados pobres solo pueden ofrecer un bajo nivel de atención a la población local. El gobierno provincial proporciona subsidios del lado de la oferta a los hospitales.",3,476,gasto,1513,1,"[dos mil siete, gobierno, central, financio, directamente, solo, 03, gasto, total, salud, realizo, transferencias, especificas, gasto, salud, ascendieron, 56, gasto, total, mayor, parte, gasto, presupuestario, salud, realizan, gobiernos, condados, condados, pobres, solo, pueden, ofrecer, bajo, nivel, atencion, poblacion, local, gobierno, provincial, proporciona, subsidios, lado, oferta, hospitales]"
2,"Claramente, hay muchos otros factores en juego, en particular, una variedad de barreras y determinantes sociales. Estos pueden estar relacionados con el género, la clase, la etnia, la casta, el idioma y la religión, y surgen de estructuras, normas y procesos sociales arraigados que aceptan e incluso fomentan distribuciones injustas de la riqueza y los recursos sociales. Por ejemplo, las personas que viven con el VIH y algunas otras enfermedades crónicas enfrentan altos niveles de estigma y discriminación que dificultan su acceso a pruebas médicas, tratamiento, atención y apoyo. Por lo tanto, los gobiernos que buscan mejorar los estándares de salud querrán mejorar la condición de la mujer, brindar servicios de salud reproductiva efectivos, implementar programas integrales de lactancia materna en línea con las recomendaciones actuales y aumentar las presiones comunitarias contra el aborto selectivo por sexo. Las personas pobres, que carecen de educación e información, o de fondos para opciones más saludables, son más vulnerables al impacto de las ENT que los ricos y es probable que mueran antes. Directa e indirectamente, las ENT tendrán impactos de gran alcance en el progreso hacia los ODM.",3,1207,de,1513,1,"[claramente, factores, j

In [119]:
#Normalización
stemmer = SnowballStemmer('spanish')



In [120]:

def stem_words(words):
    """Stem words in list of tokenized words using NLTK"""
    return [stemmer.stem(word) for word in words]


In [121]:
#!pip install spacy
!python -m spacy download es_core_news_sm

     ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
     --------------------------------------- 0.0/12.9 MB 320.0 kB/s eta 0:00:41
     --------------------------------------- 0.1/12.9 MB 762.6 kB/s eta 0:00:17
     -- ------------------------------------- 0.8/12.9 MB 5.7 MB/s eta 0:00:03
     ---------- ----------------------------- 3.5/12.9 MB 18.5 MB/s eta 0:00:01
     --------------------- ------------------ 6.8/12.9 MB 28.8 MB/s eta 0:00:01
     ------------------------------ -------- 10.0/12.9 MB 35.5 MB/s eta 0:00:01
     --------------------------------------  12.8/12.9 MB 65.6 MB/s eta 0:00:01
     --------------------------------------- 12.9/12.9 MB 50.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [122]:
import spacy

# Inicializar el modelo de spaCy para español
nlp = spacy.load('es_core_news_sm')

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words using spaCy for Spanish"""
    doc = nlp(" ".join(words))
    return [token.lemma_ if token.pos_ == 'VERB' else token.text for token in doc]

In [123]:
def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

In [124]:
textos['words'] = textos['words'].apply(stem_and_lemmatize) #Aplica lematización y Eliminación de Prefijos y Sufijos.
textos.head()

,Textos_espanol,sdg,Conteo,Moda,Max,Min,words
0,"Por ejemplo, el nÃºmero de consultas externas de especialistas es de 319 por cada mil derechohabientes en el SP, en comparaciÃ³n con 338 y 620 por cada mil derechohabientes en el IMSS y el ISSSTE, respectivamente. Si bien algunas de estas diferencias pueden reflejar una necesidad desigual (como la poblaciÃ³n ligeramente mayor del ISSSTE), otras no pueden justificarse de esta manera. El nÃºmero de recetas que no pudieron ser surtidas en su totalidad por un farmacÃ©utico debido a la falta de existencias es de 33% dentro del SP en comparaciÃ³n con 14% dentro del IMSS segÃºn los datos de la encuesta (aunque las propias cifras de los institutos de la SS sugieren tasas mÃ¡s altas de recetas surtidas). Ambas cifras se encuentran entre las mÃ¡s altas de la OCDE. El gasto de bolsillo no se ha reducido significativamente en la última década, a pesar de los esfuerzos para lograr la cobertura sanitaria universal a través de la reforma del SP.",3,944,de,1513,1,"[ejempl, naomer, consult, extern, especial, trescientos diecinuev, cad, mil, derechohabient, sp, comparacia3n, trescientos treinta y och, seiscientos veint, cad, mil, derechohabient, imss, issste, respect, si, bien, diferent, pued, reflej, neces, desigual, poblacia3n, liger, mayor, issste, pued, justific, maner, naomer, recet, pud, ser, surt, total, farmacaut, deb, falt, existent, treinta y tr, dentr, sp, comparacia3n, catorc, dentr, imss, segaon, dat, encuest, aunqu, propi, cifr, institut, ss, sugier, tas, mas, altas, recet, surt, ambas, cifr, encuentr, mas, altas, ocde, gast, bolsill, reduc, signific, ultim, dec, pes, esfuerz, logr, cobertur, sanitari, universal, trav, reform, sp, ejemplo, naomero, consultas, externas, especialistas, trescientos, diecinueve, cada, mil, derechohabientes, sp, comparacia3n, trescientos, treinta, y, ...]"
1,"En 2007, el gobierno central financió directamente solo el 0,3% del gasto total en salud, pero realizó transferencias específicas para el gasto en salud que ascendieron a otro 5,6% del gasto total. La mayor parte del gasto presupuestario en salud lo realizan los gobiernos de los condados. Por lo tanto, los condados pobres solo pueden ofrecer un bajo nivel de atención a la población local. El gobierno provincial proporciona subsidios del lado de la oferta a los hospitales.",3,476,gasto,1513,1,"[dos mil siet, gobiern, central, financi, direct, sol, 03, gast, total, salud, realiz, transferent, especif, gast, salud, ascend, 56, gast, total, mayor, part, gast, presupuestari, salud, realiz, gobi, cond, cond, pobr, sol, pued, ofrec, baj, nivel, atencion, poblacion, local, gobiern, provincial, proporcion, subsidi, lad, ofert, hospital, dos, mil, siete, gobierno, central, financio, directamente, solo, 03, gasto, total, salud, realizo, transferencias, especificas, gasto, salud, ascender, 56, gasto, total, mayor, parte, gasto, presupuestario, salud, realizar, gobiernos, condados, condados, pobres, solo, pueden, ofrecer, bajo, nivel, atencion, poblacion, local, gobierno, provincial, proporcionar, subsidios, lado, oferta, hospitales]"
2,"Claramente, hay muchos otros factores en juego, en particular, una variedad de barreras y determinantes sociales. Estos pueden estar relacionados con el género, la clase, la etnia, la casta, el idioma y la religión, y surgen de estructuras, normas y procesos sociales arraigados que aceptan e incluso fomentan distribuciones injustas de la riqueza y los recursos sociales. Por ejemplo, las personas que viven con el VIH y algunas otras enfermedades crónicas enfrentan altos niveles de estigma y discriminación que dificultan su acceso a pruebas médicas, tratamiento, atención y apoyo. Por lo tanto, los gobiernos que buscan mejorar los estándares de salud querrán mejorar la condición de la mujer, brindar servicios de salud reproductiva efectivos, implementar programas integrales de lactancia materna en línea con las recomendaciones actuales y aumentar las presiones comunitarias contra e

In [125]:
#Separacion variable predictora
textos['words'] = textos['words'].apply(lambda x: ' '.join(map(str, x)))
textos.head()

,Textos_espanol,sdg,Conteo,Moda,Max,Min,words
0,"Por ejemplo, el nÃºmero de consultas externas de especialistas es de 319 por cada mil derechohabientes en el SP, en comparaciÃ³n con 338 y 620 por cada mil derechohabientes en el IMSS y el ISSSTE, respectivamente. Si bien algunas de estas diferencias pueden reflejar una necesidad desigual (como la poblaciÃ³n ligeramente mayor del ISSSTE), otras no pueden justificarse de esta manera. El nÃºmero de recetas que no pudieron ser surtidas en su totalidad por un farmacÃ©utico debido a la falta de existencias es de 33% dentro del SP en comparaciÃ³n con 14% dentro del IMSS segÃºn los datos de la encuesta (aunque las propias cifras de los institutos de la SS sugieren tasas mÃ¡s altas de recetas surtidas). Ambas cifras se encuentran entre las mÃ¡s altas de la OCDE. El gasto de bolsillo no se ha reducido significativamente en la última década, a pesar de los esfuerzos para lograr la cobertura sanitaria universal a través de la reforma del SP.",3,944,de,1513,1,ejempl naomer consult extern especial trescientos diecinuev cad mil derechohabient sp comparacia3n trescientos treinta y och seiscientos veint cad mil derechohabient imss issste respect si bien diferent pued reflej neces desigual poblacia3n liger mayor issste pued justific maner naomer recet pud ser surt total farmacaut deb falt existent treinta y tr dentr sp comparacia3n catorc dentr imss segaon dat encuest aunqu propi cifr institut ss sugier tas mas altas recet surt ambas cifr encuentr mas altas ocde gast bolsill reduc signific ultim dec pes esfuerz logr cobertur sanitari universal trav reform sp ejemplo naomero consultas externas especialistas trescientos diecinueve cada mil derechohabientes sp comparacia3n trescientos treinta y ocho seiscientos veinte cada mil derechohabientes imss issstar respectivamente si bien diferencias pueden reflejar necesidad desigual poblacia3n ligeramente mayor issste pueden justificar él manera naomero recetas pudieron ser surtir totalidad farmacautico debido faltar existencias treinta y tres dentro sp comparacia3n catorce dentro imss segaon datos encuesta aunque propias cifras institutos ss sugerir tasas mas altas recetas surtidas ambas cifras encontrar mas altas ocde gasto bolsillo reducido significativamente ultima decada pesar esfuerzos lograr cobertura sanitaria universal traves reforma sp
1,"En 2007, el gobierno central financió directamente solo el 0,3% del gasto total en salud, pero realizó transferencias específicas para el gasto en salud que ascendieron a otro 5,6% del gasto total. La mayor parte del gasto presupuestario en salud lo realizan los gobiernos de los condados. Por lo tanto, los condados pobres solo pueden ofrecer un bajo nivel de atención a la población local. El gobierno provincial proporciona subsidios del lado de la oferta a los hospitales.",3,476,gasto,1513,1,dos mil siet gobiern central financi direct sol 03 gast total salud realiz transferent especif gast salud ascend 56 gast total mayor part gast presupuestari salud realiz gobi cond cond pobr sol pued ofrec baj nivel atencion poblacion local gobiern provincial proporcion subsidi lad ofert hospital dos mil siete gobierno central financio directamente solo 03 gasto total salud realizo transferencias especificas gasto salud ascender 56 gasto total mayor parte gasto presupuestario salud realizar gobiernos condados condados pobres solo pueden ofrecer bajo nivel atencion poblacion local gobierno provincial proporcionar subsidios lado oferta hospitales
2,"Claramente, hay muchos otros factores en juego, en particular, una variedad de barreras y determinantes sociales. Estos pueden estar relacionados con el género, la clase, la etnia, la casta, el idioma y la religión, y surgen de estructuras, normas y procesos sociales arraigados que aceptan e incluso fomentan distribuciones injustas de la riqueza y los recursos sociales. Por ejemplo, las personas que viven con el VIH y algunas otras enfermedades crónicas enfrentan altos niveles de e

In [126]:
X_data, y_data = textos['words'],textos['sdg']
y_data

0       3
1       3
2       3
3       3
4       3
       ..
2995    5
2996    5
2997    5
2998    5
2999    5
Name: sdg, Length: 3000, dtype: int64

Opcion 1

In [127]:
# Se transforma el texto con una transformación binaria 1 a 1
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(X_data)
print(X_dummy.shape)
X_dummy.toarray()[0]

(3000, 21877)


array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

Opcion 2

In [128]:
# se puede realizar el conteo de las palabras presentes
count = CountVectorizer()
X_count = count.fit_transform(X_data)
print(X_count.shape)
X_count.toarray()[0]

(3000, 21877)


array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

### APLICACIÓN DEL MODELO
Aplicaremos 3 algoritmos de clasificación para texto como se planteo como de propuesta en la documentación

## Modelamiento con arboles de decicion

In [129]:
# Importamos lo necesario para este algoritmo
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold 

In [130]:
# Dividimos los datos en entrenamiento y test
X = X_dummy
Y = y_data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [131]:
# buscamos los hiperparametros 
particiones = KFold(n_splits=5, shuffle=True, random_state = 0)
param_grid={'criterion': ['gini', 'entropy'],'max_depth': [2, 4, 6, 8, 10, 20, 30, 50],'min_samples_split': [2, 3, 4, 5, 6, 7]}

In [132]:
#Arbol sin parametros
arbol = DecisionTreeClassifier(random_state=0)

In [133]:
# Busacamos el mejor modelo con GridSearch
best_model = GridSearchCV(arbol, param_grid, cv=particiones)
# Ajuste del modelo
best_model.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=0),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 6, 8, 10, 20, 30, 50],
                         'min_samples_split': [2, 3, 4, 5, 6, 7]})

In [134]:
#Resultado de la busqueda
best_model.best_params_

{'criterion': 'gini', 'max_depth': 30, 'min_samples_split': 2}

In [135]:
#Construccion del arbol con los mejores parametros
best_tree = best_model.best_estimator_
y_pred_train = best_tree.predict(X_train)
y_pred_test = best_tree.predict(X_test)
print('Exactitud sobre entrenamiento: %.2f' % accuracy_score(Y_train, y_pred_train))
print('Exactitud sobre test: %.2f' % accuracy_score(Y_test, y_pred_test))

Exactitud sobre entrenamiento: 1.00
Exactitud sobre test: 0.93


In [136]:
# Se genera la matriz de confusión
cm = confusion_matrix(Y_test, y_pred_test)
cm

array([[219,   3,  15],
       [  7, 174,   6],
       [  6,   6, 164]], dtype=int64)

In [137]:
# Se puede visualizar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=best_tree.classes_)
disp.plot()
plt.savefig("ConfusionMatrixnDecisionTreeClassifier.png")
plt.show()

C:\Users\oipli\AppData\Local\Temp\ipykernel_33760\2317007602.py:5: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [138]:
# Mostrar reporte de clasificación
print(classification_report(Y_test, y_pred_test))

              precision    recall  f1-score   support

           3       0.94      0.92      0.93       237
           4       0.95      0.93      0.94       187
           5       0.89      0.93      0.91       176

    accuracy                           0.93       600
   macro avg       0.93      0.93      0.93       600
weighted avg       0.93      0.93      0.93       600



Notamos que tenemos muy buenos puntajes, ya que se clasifican correctamente en promedio el 93% de los textos